In [1]:
import pandas as pd
import sqlalchemy
import math
import os, sys, copy, random, pickle, cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append("../")

%load_ext autoreload
%autoreload 2

from modules.ControlTracking import TrackFactory
from modules.ControlDisplay import Display
from modules.Interproc import DBInterface, GuiviewState

from modules.AnalysisHelpers import ( 
         exploreImgs, buildImgComparisonData, subprocEval)

from modules.EvalHelpers import EvalTracker, EvalDataset
from modules.ControlEval import EvalFactory
from modules.DataSchemas import ScoreSchema
from modules.EvalHelpers import OutcomeData, DFHelper

from modules.EvalHelpers import AggEval, DFHelper

In [3]:
tmp_db_path = 'data/misc/books/eval-module-4/evaldb.db'

vid_path = 'data/proc/tmp/dec14/output5.proc1.proc1.avi'

outcome_subproc = subprocEval(f_pathfn = vid_path
                              ,db_pathfn = tmp_db_path 
                              ,b_log = True
                              ,b_eval_log = True
                             )


START - running eval module...
[                            ]
-----------------------------
output db: data/misc/books/eval-module-4/evaldb.db
output tbl: outcome_dataframe
eval time: 4.287
FINISH - rows: 345 cols: 48 

num frames:                  345
obj enums scored/tracked:    0,1 / 0
num scored frames:           19 | 0,9,78...308,324,340
-------
agg_checkBaselineInsideTrack       0.42
agg_checkBothContainsOther         0.42
agg_checkEitherContainsOther       0.53
agg_checkTrackInsideBaseline       0.53
agg_checkTrackInsideBaselineRect   0.11
agg_checkTrackSuccess              1.00
agg_compareRadii                   2.32
agg_distanceFromBaseline           5.49
subproc time: 5.952
